# --- Day 16: Proboscidea Volcanium ---

https://adventofcode.com/2022/day/16

## Get Input Data

In [1]:
import re

def get_data(filename):
    """Get input data for puzzle.
    
    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.
    
    Returns
    -------
    graph : dict
    flow_rates : dict
    """
    graph = {}
    flow_rates = {}

    pattern = r"Valve ([A-Z]{2}) .+=(\d+); .+ valves? (.+)"
    file_str = open(f"../inputs/{filename}.txt").read()

    for node, flow_rate, neighbors in re.findall(pattern, file_str):
        graph[node] = {}
        for neighbor in neighbors.split(", "):
            graph[node][neighbor] = 1
                
        flow_rates[node] = int(flow_rate)

    return graph, flow_rates

In [2]:
test_valves_graph, test_flow_rates = get_data("test_valves")
test_valves_graph

{'AA': {'DD': 1, 'II': 1, 'BB': 1},
 'BB': {'CC': 1, 'AA': 1},
 'CC': {'DD': 1, 'BB': 1},
 'DD': {'CC': 1, 'AA': 1, 'EE': 1},
 'EE': {'FF': 1, 'DD': 1},
 'FF': {'EE': 1, 'GG': 1},
 'GG': {'FF': 1, 'HH': 1},
 'HH': {'GG': 1},
 'II': {'AA': 1, 'JJ': 1},
 'JJ': {'II': 1}}

In [3]:
test_flow_rates

{'AA': 0,
 'BB': 13,
 'CC': 2,
 'DD': 20,
 'EE': 3,
 'FF': 0,
 'GG': 0,
 'HH': 22,
 'II': 0,
 'JJ': 21}

In [4]:
test_worthwhile_valves = [v for v in test_flow_rates if test_flow_rates[v] > 0]
test_worthwhile_valves

['BB', 'CC', 'DD', 'EE', 'HH', 'JJ']

In [5]:
valves_graph, flow_rates = get_data("valves")

In [6]:
[[k, valves_graph[k]] for k in list(valves_graph.keys())[:10]]

[['XB', {'WZ': 1, 'LE': 1}],
 ['BM', {'PL': 1, 'RI': 1}],
 ['GC', {'HN': 1, 'IT': 1}],
 ['RM', {'ZQ': 1, 'YL': 1}],
 ['ZM', {'SN': 1, 'KE': 1, 'UW': 1, 'MY': 1, 'GW': 1}],
 ['UH', {'HM': 1, 'HN': 1}],
 ['GW', {'LE': 1, 'ZM': 1}],
 ['HN', {'UW': 1, 'UH': 1, 'GL': 1, 'WZ': 1, 'GC': 1}],
 ['VT', {'ZD': 1, 'PE': 1}],
 ['VI', {'JS': 1, 'AA': 1}]]

In [7]:
flow_rates

{'XB': 0,
 'BM': 0,
 'GC': 0,
 'RM': 0,
 'ZM': 5,
 'UH': 0,
 'GW': 0,
 'HN': 19,
 'VT': 0,
 'VI': 0,
 'YL': 12,
 'LA': 0,
 'CM': 0,
 'JI': 24,
 'ZD': 25,
 'VB': 0,
 'FO': 0,
 'JS': 0,
 'RI': 0,
 'XD': 14,
 'ES': 11,
 'WZ': 0,
 'HW': 0,
 'KE': 0,
 'IY': 22,
 'XW': 0,
 'BE': 0,
 'QY': 23,
 'MH': 0,
 'IT': 0,
 'DA': 0,
 'PM': 0,
 'WU': 0,
 'UL': 0,
 'SM': 13,
 'XC': 0,
 'OJ': 0,
 'SN': 0,
 'WH': 0,
 'UW': 0,
 'HM': 0,
 'AB': 0,
 'QR': 8,
 'UV': 0,
 'ZQ': 20,
 'ZJ': 0,
 'GL': 0,
 'CP': 0,
 'AM': 0,
 'PL': 17,
 'YE': 0,
 'PE': 4,
 'MY': 0,
 'QX': 0,
 'GI': 0,
 'WW': 0,
 'FF': 0,
 'LE': 6,
 'GU': 0,
 'AA': 0}

In [21]:
worthwhile_valves = [v for v in flow_rates if flow_rates[v] > 0]
worthwhile_valves

['ZM',
 'HN',
 'YL',
 'JI',
 'ZD',
 'XD',
 'ES',
 'IY',
 'QY',
 'SM',
 'QR',
 'ZQ',
 'PL',
 'PE',
 'LE']

## Part 1
---

What is the decision to make at each minute?

These are the state variables:
1. `current_valve` : Where in the graph we are
2. `time_left` : Countdown in minutes from 30
3. `open_valves` : An array keeping track of which of the `worthwhile_valves` are open (order matters, but that's captured in the `pressure_release`)
4. `closed_valves` : An array keeyping track of which of the `worthwhile_vales` are still closed
5. `pressure_release` : The amount of pressure that would be released if 

At each minute, have to decide:
* Open current valve?
* Move to another valve?
* Stay put? (All `worthwhile_valves` are open already, so there's nothing worth doing)

While the test data is relatively small, the actual input data is significantly larger.  
The test data have only 6 `worthwhile_valves`, but the actual input data have 15 `worthwhile_valves`.

While that doesn't seem like a big difference, it is!

That's because the number of possible permutations explodes as $n$ increases.  
The number of permutations of an array with $n$ elements is $n!$

`math.factorial(6)` is 720  
`math.factorial(15)` is ~1.3 trillion





In [8]:
import math
print(f" 6! is equal to: {math.factorial(6)}")
print(f"15! is equal to: {math.factorial(15)}")

 6! is equal to: 720
15! is equal to: 1307674368000


In [9]:
from collections import defaultdict
from itertools import product

def fw(graph):
    """See algorithms/floyd_warshall.ipynb for comments/details."""
    dist = defaultdict(lambda: float("inf"))

    for node, neighbors in graph.items():
        dist[(node, node)] = 0
        for neighbor, edge_weight in neighbors.items():
            dist[(node, neighbor)] = edge_weight

    for k, i, j in product(graph.keys(), repeat=3):
        dist[(i, j)] = min(dist[(i, j)], dist[(i, k)] + dist[(k, j)])

    return dict(dist)

In [10]:
test_min_dists = fw(test_valves_graph)
[(v, test_min_dists[v]) for v in list(test_min_dists.keys())[:5]]

[(('AA', 'AA'), 0),
 (('AA', 'DD'), 1),
 (('AA', 'II'), 1),
 (('AA', 'BB'), 1),
 (('BB', 'BB'), 0)]

In [11]:
min_dists = fw(valves_graph)
[(v, min_dists[v]) for v in list(min_dists.keys())[:5]]

[(('XB', 'XB'), 0),
 (('XB', 'WZ'), 1),
 (('XB', 'LE'), 1),
 (('BM', 'BM'), 0),
 (('BM', 'PL'), 1)]

In [12]:
from functools import cache

In [56]:
# @cache
def find_all_perms(curr_valve, time_left=30, perm='', curr_release=0):

    perms = []
    perm += curr_valve
    perms.append((perm, curr_release))
    # perms.append(curr_release)

    print("foo", perm, time_left, test_flow_rates[curr_valve], curr_release)

    # Base case
    if time_left <= 2:
        # return [(perm, curr_release)]
        # return [curr_release]
        return perms

    for valve in ["BB", "CC",]: #test_worthwhile_valves[:3]:
        if valve not in perm:
            time_left -= (min_dists[(curr_valve, valve)] + 1)
            curr_release += time_left * test_flow_rates[valve]
            print("bar", perm, time_left, test_flow_rates[valve], curr_release)

            # perms += find_all_perms(valve, time_left, perm)
            perms += find_all_perms(valve, time_left, perm, curr_release)
            # perms.append((perm, curr_release))
            # perms.append(curr_release)

    return perms

In [45]:
perm = "asweudqeocjr"
[perm[i:i+2] for i in range(0, int(len(perm)), 2)]

['as', 'we', 'ud', 'qe', 'oc', 'jr']

In [48]:
# THIS ONE WORKS!

@cache
def find_all_perms(curr_valve, time_left=30, perm='', release=0):

    perms = []
    perm += curr_valve

    if len(perm) > 2:
        prev_valve = perm[-4:-2]
        time_left -= (min_dists[(prev_valve, curr_valve)] + 1)

    release += time_left * flow_rates[curr_valve]
    perms.append(release)

    # Base case
    if time_left <= 2:
        return perms

    # perms.append((perm, curr_release))
    # perms.append(release)

    # print("foo", perm, time_left, test_flow_rates[curr_valve], curr_release)

    for valve in worthwhile_valves: #test_worthwhile_valves:
        if valve not in [perm[i:i+2] for i in range(0, int(len(perm)), 2)]: #perm:
            # time_left -= (test_min_dists[(curr_valve, valve)] + 1)
            # release += time_left * test_flow_rates[valve]
            # print("bar", perm, time_left, test_flow_rates[valve], curr_release)

            # perms += find_all_perms(valve, time_left, perm)
            perms += find_all_perms(valve, time_left, perm, release)
            # perms.append((perm, curr_release))


    return perms


In [49]:
max(find_all_perms("AA"))

2265

In [12]:
test_flow_rates

{'AA': 0,
 'BB': 13,
 'CC': 2,
 'DD': 20,
 'EE': 3,
 'FF': 0,
 'GG': 0,
 'HH': 22,
 'II': 0,
 'JJ': 21}

5040

In [75]:
import numpy as np  # import this to access np.argmax() instead
cache = {}

def max_cum_pressure_release(flow_rates, mins_left=30, start_valve="AA"):
    flow_rates = flow_rates.copy()
    cum_pressure_released = 0
    valve = start_valve

    while mins_left > 0:
        print(f"== Minutes left {mins_left} ==")

        open_valves = [k for k in flow_rates if flow_rates[k] > 0]

        if len(open_valves) > 0:
            net_benefits = []

            for open_valve in open_valves:
                cost = shortest_paths[open_valve] + 1
                benefit = (mins_left - cost) * flow_rates[open_valve]
                net_benefits.append(benefit - cost)

                print(valve, open_valve, cost, benefit, benefit-cost)

            next_valve = open_valves[np.argmax(net_benefits)]

            valve = next_valve
            cost = shortest_paths[next_valve]

            cum_pressure_released += (mins_left - cost - 1) * flow_rates[next_valve]
            flow_rates[next_valve] = 0
            mins_left -= cost

        else:
            # Just chill -- all the valves are already open
            mins_left -= 1

        print(f"Current valve: {valve}")
        print(f"Cumulative pressure released: {cum_pressure_released}")

    return cum_pressure_released

### Run on Test Data

In [76]:
max_cum_pressure_release(test_valves_graph, test_flow_rates) # == 1651

== Minutes left 30 ==
AA BB 2 364 362
AA CC 3 54 51
AA DD 2 560 558
AA EE 3 81 78
AA HH 6 528 522
AA JJ 3 567 564
Current valve: JJ
Cumulative pressure released: 567
== Minutes left 28 ==
JJ BB 4 312 308
JJ CC 5 46 41
JJ DD 4 480 476
JJ EE 5 69 64
JJ HH 8 440 432
Current valve: DD
Cumulative pressure released: 1047
== Minutes left 25 ==
DD BB 3 286 283
DD CC 2 46 44
DD EE 2 69 67
DD HH 5 440 435
Current valve: HH
Cumulative pressure released: 1487
== Minutes left 21 ==
HH BB 7 182 175
HH CC 6 30 24
HH EE 4 51 47
Current valve: BB
Cumulative pressure released: 1669
== Minutes left 15 ==
BB CC 2 26 24
BB EE 4 33 29
Current valve: EE
Cumulative pressure released: 1702
== Minutes left 12 ==
EE CC 3 18 15
Current valve: CC
Cumulative pressure released: 1720
== Minutes left 10 ==
Current valve: CC
Cumulative pressure released: 1720
== Minutes left 9 ==
Current valve: CC
Cumulative pressure released: 1720
== Minutes left 8 ==
Current valve: CC
Cumulative pressure released: 1720
== Minutes lef

1720

### Run on Input Data

## Part 2
---

### Run on Test Data

### Run on Input Data